# Teil 2: Einleitung zum Federated Learning

Im letzten Abschnitt wurden PointerTensoren, welche die Infrastruktur für vertrauliches Deep Learning bilden, vorgestellt. In diesem Abschnitt erfahren Sie, wie Sie diese nutzen und Ihren ersten Algorithmus fürs vertrauliche Deep Learning implementieren: Federated Learning.

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

### Was ist Federated Learning?

Es ist eine einfache und mächtige Art und Weise Deep Learning Modele zu trainieren.  
Wenn Sie an Trainings-Datensätze denken, dann sind diese immer auch ein Ergebnis eines Sammelprozesses. Menschen (mit ihren Geräten) generieren Daten indem sie Ereignisse in der realen Welt beobachten. Normalerweise werden diese Daten anschließend an einem einzigen, zentralen Ort gesammelt, um anschließend ein Machine Learning Model auf ihnen zu trainieren. Federated Learning dreht diesen Prozess jedoch um!

Anstatt die Trainingsdaten zum Model (einem zentralen Server) zu bringen, wird das Model zu den Geräten mit den Trainingsdaten gebracht. 

Die Idee dahinter ist, dass der Besitzer der Daten die einzige existierende Kopie der Daten behält und er damit selbst entscheiden kann, wem er Zugang gewährt. Ziemlich cool, nicht wahr?

# Abschnitt 2.1 - Ein einfaches Federated Learning Beispiel

Lassen Sie uns mit dem dezentralen Trainieren eines einfachen Models starten. Das Model wird dabei so grundlegend wie möglich sein.  
Dazu brauchen wir:

- einen einfachen Datensatz
- ein Model
- grundlegende Trainings Infrastruktur zum Anpassen des Models an die Daten

Anmerkung:  
Wenn diese API Ihnen nicht geläufig ist - können Sie unter [fast.ai](http://fast.ai) einen Kurs absolvieren bevor Sie dieses Tutorial fortführen.

In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [ ]:
train()

So weit, so gut! Wir haben ein einfaches Model auf die konventionelle Art trainiert. Alle unsere Daten liegen auf unserer lokalen Maschine vor und wir können diese nutzen für die Updates des Models. Federated Learning funktioniert jedoch nicht auf diese Weise.  
Lassen Sie uns das Beispiel für das Federated Learning anpassen!

Dazu müssen wir:

- einige Helfer erstellen
- Pointer zu den Trainingsdaten auf jedem Helfer erhalten
- die Trainings Infrastruktur aufs Federated Learning anpassen

    Neue Trainings Schritte:
    - sende das Model zum richtigen Helfer
    - trainiere auf den Daten des Helfers
    - hole das verbesserte Model zurück und wiederhole alles mit dem nächsten Helfer

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)

In [ ]:
# create a couple workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [ ]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [ ]:
def train():
    # Training Logic
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
            #print(data.location.id)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [ ]:
train()

## Gut gemacht!

Und voilà! Sie haben ein sehr einfaches Deep Learning Model mittels Federated Learning trainiert! Dabei wurde das Model zu jedem Helfer gesendet, neue Gradienten ermittelt, diese zum Verbessern des Models verwendet und das Model anschließend wieder zum lokalen Server transferiert. Zu keinem Zeitpunkt konnten die zugrundeliegenden Daten dabei direkt eingesehen werden! Die Privatsphäre von Bob und Alice wurde demnach gewahrt!!!

## Defizite von diesem Beispiel

Auch wenn dieses Beispiel eine gute Einleitung ins Federated Learning darstellt, so hat es doch große Defizite. Vor allem mit dem Aufrufen von `model.get()` und dem Erhalten des verbesserten Models von Bob oder Alice, lässt sich über die Gradienten sehr viel über die Trainingsdaten herausfinden. In einigen Fällen lassen sich die Trainingsdaten sogar komplett wieder herstellen!

Wie lässt sich dem entgegenwirken? Die erste Strategie, die häufig eingesetzt wird, ist **über die Gradienten mehrerer Individuen zu mitteln bevor sie als Update zum zentralen Server geschickt werden**. Diese Strategie benötigt jedoch einige anspruchsvollere PointerTensor-Objekte. Deshalb geht es im nächsten Abschnitt um fortgeschrittenere Pointer Funktionalitäten und anschließend wird dieses Federated Learning Beispiel angepasst.

# Herzlichen Glückwunsch!!! - Zeit, der Community beizutreten! 

Herzlichen Glückwunsch zum Abschluss dieses Notebook-Tutorials! Wenn es Ihnen gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralisiertenen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun! 

### PySyft auf GitHub einen Stern geben! 

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos mit Sternen auszuzeichnen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen. 

- [Gib PySyft einen Stern](https://github.com/OpenMined/PySyft)

### Mach mit bei Slack! 

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei! 

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Entwickler zu werden! Sie können jederzeit zur PySyft GitHub Issues-Seite gehen und nach "Projects" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "good first issue" gekennzeichnet sind. 

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebase beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups! 

 - [OpenMined's Open Collective Page](https://opencollective.com/openmined)